## Gerando arquivo para NER-QA-NestedClinBr
Deixar sempre o arquivo de test para fazer o teste completo (com o pipeline todo)
1o. -> passar pelo modelo de NER normal (binario), pegar possiveis entidades
2o. -> combinacao de palavras, modelo de span

Descontinuas: treinar duplicando as frases, tirando os termos do meio.. na hora da predicao: se duas entidades estão mito proximas (ou no mesmo chunk), tirar o espaço e ver se são (ou tentar achar uma regra, ex sempre q tem tratamento e uma anotmia perto, usar uma janela.. ex duas palavras de distancia)...

Para tese: testar tbm sem as descontinuas.. pra pode comparar com outros metodos...

v2 - com corpus original Tempclinbr (arq .txt), contendo as quebras de linha.. aqui quebrar cada frase do texto em uma entrada no dicionario.

v3 - split de train em 75% para dev.. conforme feito no TempClinBr.. e tbm tirar os 41 primeiros tokens dos textos: 
'Data de Criação do Documento: 22/04/2014' como codigo Joao

In [1]:
# arrumar * e exclamacao, nao pode estar grudado na palavra pois tokenizador separa de qquer forma

In [2]:
import os
from pathlib import Path
import re
import pickle
import random


In [3]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

In [4]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [5]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None


In [6]:
pastasCorpus = [r'corpus\test',r'corpus\train']
#pastasCorpus = [r'corpus\test']
#pastasCorpus = [r'corpus\TESTE']

In [7]:
import re

s = 'Make the World a 2.4Better Place2.0'
pattern = r'([0-9])\.([0-9])'
replacement = r'\1y\2'
html = re.sub(pattern, replacement, s)

print(html)

Make the World a 2y4Better Place2y0


In [8]:
def getDicSentences(pastaCorpus):
    #devePrintar=True
    devePrintar=False
    dic_sentences = {}
    numMaxTokensPorFrase=0 # numero tokens da maior frase
    num=0
    listaEntidades=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            if devePrintar:
                print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
                pass
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                if devePrintar:
                    print('linha:', linha)
                    pass
                frases=[]
                allFrasesString=''
                numL=0
                for l in linha:
                    numL=numL+1
                    allFrasesString = allFrasesString+l
                    if numL==1: # descarta primeira frase, que é data de criação do doc
                        #print('descartando frase:', l)
                        continue
                    if l.strip() and l.strip()!='\n':
                        #print('l:', l)
                        pattern = r'([0-9])\.([0-9])'
                        replacement = r'\1==\2'
                        novoL = re.sub(pattern, replacement, l.strip())
                        l2 = novoL.split('.') # quebrando frases
                        for l3 in l2:
                            if l3.strip() and l3.strip()!='\n':
                                novaFrase = l3.replace('\n','').replace('==','.').strip()+'.'
                                frases.append(novaFrase)
                #print('frases:', frases)
                # para cada frase
                # para tokenizar nesses tokens
                #print('allFrasesString:', allFrasesString)
                frasesTokens={}
                #numCaracteresTotal=42 # primeira frase ignorada
                numIndiceAnterior=0
                for frase in frases:
                    tokens=[]
                    frase2 = frase.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace("!"," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ").replace("*"," * ")
                    frase2 = replaceWhiteSpaces(frase2)
                    frase2 = frase2.split()
                    for numtoken, token in enumerate(frase2):
                        if devePrintar:
                            print('token:', token)
                            print('numIndiceAnterior:', numIndiceAnterior)
                        if token!='.':
                            numCaracteresTotal = allFrasesString.find(token, numIndiceAnterior, len(allFrasesString))
                        else:
                            numCaracteresTotal=numIndiceAnterior+1
                        #tokens.append([token,numtoken])
                        tokens.append([token,numtoken, numCaracteresTotal])
                        numIndiceAnterior = numCaracteresTotal+len(token)-1
                        if numMaxTokensPorFrase<len(tokens):
                            numMaxTokensPorFrase = len(tokens)
                    frasesTokens[frase]=tokens
                    #linhaTokens=linha.copy()    
                if devePrintar:
                    print('frasesTokens:', frasesTokens)
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # tem descontinua?
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua, linha: {}, num: {}'.format(linha, num))
                        #print('descontinua, file: {}, num: {}'.format(filename, num))
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        #print('aaaaaaaaaaaaaaaaa')
                        dicEntidades[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio1), int(fim1)):", (int(inicio1), int(fim1)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        
                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio2), int(fim2)):", (int(inicio2), int(fim2)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))


                #print('frasesTokens:', frasesTokens)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                listaIndicesJaUsados = []
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                #print(indicesDic)
                #print('dicEntidades:', dicEntidades)
                
                for key, value in frasesTokens.items():
                    for i in indicesDic:
                        tipo_entidade,termos_entidade = dicEntidades[i]
                        ##key (i) = indices old a comparar
                        #print('key:', key)
                        #print('value:', value)
                        #print('i:', i)
                        for token in value:
                            #print('token:', token)
                            if i[0]==token[2]:
                                novo_inicio, novo_fim = [token[1],token[1]+len(termos_entidade.split())]
                                novos_indices=[]
                                for k in range(novo_inicio,novo_fim):
                                    novos_indices.append(k)
                                listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])
                                #print("[termos_entidade, novos_indices, tipo_entidade]:", [termos_entidade, novos_indices, tipo_entidade])
                            else:
                                #print('else, i[0]:',i[0])
                                pass
                        
                    if len(value)>0:
                        #print('incluindo:', key)
                        dic_sentences[num]=[value, listaEntidades]
                        listaEntidades=[]
                        num=num+1 

        #print(num)
        #if num>318:
        #    break

        #if num>3:
        #    break
    print('numMaxTokensPorFrase:', numMaxTokensPorFrase)
    return dic_sentences, frasesComDescontinuas
            


In [9]:
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])


numMaxTokensPorFrase: 146


In [10]:
dic_sentencesTest[42]

[[['Paciente', 0, 43],
  ['em', 1, 52],
  ['tratamento', 2, 55],
  ['para', 3, 66],
  ['ICC', 4, 71],
  ['diastólica', 5, 75],
  ['há', 6, 86],
  ['cerca', 7, 89],
  ['de', 8, 95],
  ['5', 9, 98],
  ['a', 10, 100],
  ['com', 11, 102],
  ['melhora', 12, 106],
  ['importante', 13, 114],
  ['dos', 14, 125],
  ['sintomas', 15, 129],
  ['após', 16, 138],
  ['início', 17, 143],
  ['do', 18, 150],
  ['tratamento', 19, 153],
  ['.', 20, 163]],
 [['tratamento', [2], 'Tratamento'],
  ['ICC diastólica', [4, 5], 'Problema'],
  ['melhora importante dos sintomas após início do tratamento',
   [12, 13, 14, 15, 16, 17, 18, 19],
   'Problema'],
  ['tratamento', [19], 'Tratamento']]]

In [11]:
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])


numMaxTokensPorFrase: 192


In [12]:
dic_sentencesTrain[26]

[[['CAt', 0, 778],
  ['agosto', 1, 782],
  ['2013', 2, 789],
  [':', 3, 793],
  ['reestenose', 4, 795],
  ['de', 5, 806],
  ['stent', 6, 809],
  ['70', 7, 815],
  ['%', 8, 817],
  ['em', 9, 819],
  ['cd', 10, 822],
  [',', 11, 824],
  ['Oclusao', 12, 826],
  ['total', 13, 834],
  ['de', 14, 840],
  ['stent', 15, 843],
  ['em', 16, 849],
  ['ramo', 17, 852],
  ['diagonalis', 18, 857],
  [',', 19, 867],
  ['*', 20, 869],
  ['angioplastia', 21, 870],
  ['de', 22, 883],
  ['agosto', 23, 886],
  ['2013', 24, 893],
  ['de', 25, 898],
  ['cornoaria', 26, 901],
  ['direita', 27, 911],
  ['com', 28, 919],
  ['stent', 29, 923],
  ['.', 30, 928]],
 [['CAt', [0], 'Teste'],
  ['reestenose de stent 70 % em cd', [4, 5, 6, 7, 8, 9, 10], 'Problema'],
  ['cd', [10], 'Anatomia'],
  ['Oclusao total de stent em ramo diagonalis',
   [12, 13, 14, 15, 16, 17, 18],
   'Problema'],
  ['ramo diagonalis', [17, 18], 'Anatomia'],
  ['angioplastia', [21], 'Tratamento'],
  ['cornoaria direita', [26, 27], 'Anatomia']]

In [13]:
print('--Treinamento:--', pastasCorpus[1])
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])
print('len(sentences):', len(dic_sentencesTrain))
print('len(Descontinuas):',len(frasesComDescontinuasTrain))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTrain)))


print('--Teste:--', pastasCorpus[0])
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])
print('len(sentences):', len(dic_sentencesTest))
print('len(Descontinuas):',len(frasesComDescontinuasTest))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTest)))


--Treinamento:-- corpus\train
numMaxTokensPorFrase: 192
len(sentences): 1736
len(Descontinuas): 91
len(frasesComDescontinuas): 51
--Teste:-- corpus\test
numMaxTokensPorFrase: 146
len(sentences): 506
len(Descontinuas): 44
len(frasesComDescontinuas): 17


In [14]:
dic_sentencesTest[9]

[[['Abd', 0, 644],
  ['globoso', 1, 648],
  [',', 2, 655],
  ['flacido', 3, 657],
  [',', 4, 664],
  ['indolor', 5, 666],
  ['a', 6, 674],
  ['palpacao', 7, 676],
  [',', 8, 684],
  ['sem', 9, 686],
  ['VCM', 10, 690],
  ['.', 11, 693]],
 [['Abd', [0], 'Anatomia'], ['VCM', [10], 'Problema']]]

In [15]:
dic_sentencesTrain[861]

[[['CONTRAÇÃO', 0, 507],
  ['SEGMENTAR', 1, 517],
  ['DO', 2, 527],
  ['VE', 3, 530],
  ['ALTERADA', 4, 533],
  ['.', 5, 541]],
 [['CONTRAÇÃO SEGMENTAR DO VE ALTERADA', [0, 1, 2, 3, 4], 'Problema'],
  ['VE', [3], 'Anatomia']]]

In [16]:
frasesComDescontinuasTrain[:10]

[3, 15, 30, 30, 30, 30, 55, 55, 65, 65]

In [17]:
# frases com entidades descontinuas
print(len(set(frasesComDescontinuasTrain)))

51


In [18]:
frasesComDescontinuasTest

[0,
 42,
 42,
 42,
 49,
 67,
 67,
 67,
 67,
 67,
 91,
 131,
 147,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 227,
 241,
 241,
 241,
 241,
 241,
 241,
 241,
 263,
 291,
 291,
 291,
 316,
 329,
 367,
 367,
 367,
 367,
 406,
 468]

In [19]:
print(dic_sentencesTest[2])

[[['Comorbidades', 0, 142], [':', 1, 154], ['DM', 2, 156], ['há', 3, 159], ['10', 4, 162], ['anos', 5, 165], ['em', 6, 170], ['uso', 7, 173], ['de', 8, 177], ['metformina', 9, 180], ['850mg', 10, 191], ['3', 11, 197], ['cp', 12, 199], ['/', 13, 201], ['dia', 14, 202], [',', 15, 205], ['acarbose', 16, 207], ['1', 17, 216], ['cp', 18, 218], ['/', 19, 220], ['dia', 20, 221], ['e', 21, 225], ['glicazida', 22, 227], ['60mg', 23, 237], ['2', 24, 242], ['cp', 25, 244], ['/', 26, 246], ['dia', 27, 247], ['e', 28, 251], ['insulina', 29, 253], ['(', 30, 262], ['24', 31, 263], ['-', 32, 266], ['0', 33, 268], ['-', 34, 270], ['24', 35, 272], [')', 36, 274], ['.', 37, 275]], [['Comorbidades', [0], 'Problema'], ['DM', [2], 'Problema'], ['metformina 850mg', [9, 10], 'Tratamento'], ['acarbose', [16], 'Tratamento'], ['glicazida 60mg', [22, 23], 'Tratamento'], ['insulina', [29], 'Tratamento']]]


In [20]:
#save_obj('dic_sentencesTrainDev',dic_sentencesTrain)
porc=0.76
tamanhoTotal = len(dic_sentencesTrain)
tamanhoTrain = int(tamanhoTotal*porc)
print('tamanhoTrain', tamanhoTrain)
tamanhoDev = tamanhoTotal - tamanhoTrain
print('tamanhoDev', tamanhoDev)
dic_sentencesDev_temp = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[tamanhoTrain:-1]}
dic_sentencesTrain = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[:tamanhoTrain]}
num=0
dic_sentencesDev = {}
for key, value in dic_sentencesDev_temp.items():
    dic_sentencesDev[num] = value
    num=num+1

print('len(dic_sentencesTrain):', len(dic_sentencesTrain))
print('len(dic_sentencesDev):', len(dic_sentencesDev))
print(dic_sentencesTrain[tamanhoTrain-1])
print(dic_sentencesDev[0])

tamanhoTrain 1319
tamanhoDev 417
len(dic_sentencesTrain): 1319
len(dic_sentencesDev): 416
[[['Paciente', 0, 151], ['relata', 1, 160], ['apenas', 2, 167], ['um', 3, 174], ['episodio', 4, 177], ['no', 5, 186], ['momento', 6, 189], ['de', 7, 197], ['gripe', 8, 200], ['.', 9, 205]], [['gripe', [8], 'Problema']]]
[[['HAS', 0, 207], [',', 1, 210], ['ICC', 2, 212], [',', 3, 215], ['nega', 4, 217], ['DM', 5, 222], ['.', 6, 224]], [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]


In [21]:
dic_sentencesDev[0]

[[['HAS', 0, 207],
  [',', 1, 210],
  ['ICC', 2, 212],
  [',', 3, 215],
  ['nega', 4, 217],
  ['DM', 5, 222],
  ['.', 6, 224]],
 [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]

In [22]:
dic_sentencesDev[1]

[[['carvedilol', 0, 226],
  ['1', 1, 237],
  ['cp', 2, 239],
  ['12', 3, 242],
  ['/', 4, 244],
  ['12', 5, 245],
  [',', 6, 247],
  ['furosemida', 7, 249],
  ['20mg', 8, 260],
  ['2', 9, 265],
  ['cp', 10, 267],
  ['de', 11, 270],
  ['12', 12, 273],
  ['/', 13, 275],
  ['12', 14, 276],
  [',', 15, 278],
  ['sinvastatina', 16, 280],
  ['1cp', 17, 293],
  ['a', 18, 297],
  ['noite', 19, 299],
  [',', 20, 304],
  ['AAS', 21, 306],
  ['100mg', 22, 310],
  ['apos', 23, 316],
  ['almoço', 24, 321],
  ['e', 25, 328],
  ['Omeprazol', 26, 330],
  ['20mg', 27, 340],
  ['1', 28, 345],
  ['xdia', 29, 347],
  ['.', 30, 351]],
 [['carvedilol', [0], 'Tratamento'],
  ['furosemida 20mg', [7, 8], 'Tratamento'],
  ['sinvastatina', [16], 'Tratamento'],
  ['AAS 100mg', [21, 22], 'Tratamento'],
  ['Omeprazol 20mg', [26, 27], 'Tratamento']]]

In [23]:
save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesDev',dic_sentencesDev)
save_obj('dic_sentencesTest',dic_sentencesTest)

In [24]:
dic_sentencesTrain[0]

[[['Dispneia', 0, 43],
  ['importante', 1, 52],
  ['aos', 2, 63],
  ['esforços', 3, 67],
  ['+', 4, 76],
  ['dor', 5, 78],
  ['tipo', 6, 82],
  ['peso', 7, 87],
  ['no', 8, 92],
  ['peito', 9, 95],
  ['no', 10, 101],
  ['esforço', 11, 104],
  ['.', 12, 111]],
 [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'],
  ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'],
  ['peito', [9], 'Anatomia']]]

In [30]:
for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    for tipoEntidade in listaEntidades:
        indiceEnts=[]
        isEntidade=0
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            if tag!=tipoEntidade:
                tag='O'
                isEntidade=0
            else:
                if (isEntidade==1):
                    tag='I-ENT'
                else:
                    tag='B-ENT'
                isEntidade=1
                
            num_entidade_dev=num_entidade_dev+1
            print(tokenGravar, tag)

Dispneia B-ENT
importante I-ENT
aos I-ENT
esforços I-ENT
+ O
dor B-ENT
tipo I-ENT
peso I-ENT
no I-ENT
peito I-ENT
no I-ENT
esforço I-ENT
. O
Dispneia O
importante O
aos O
esforços O
+ O
dor O
tipo O
peso O
no O
peito O
no O
esforço O
. O
Dispneia O
importante O
aos O
esforços O
+ O
dor O
tipo O
peso O
no O
peito O
no O
esforço O
. O
Dispneia O
importante O
aos O
esforços O
+ O
dor O
tipo O
peso O
no O
peito B-ENT
no O
esforço O
. O
Obeso B-ENT
, O
has B-ENT
, O
icc B-ENT
. O
Obeso O
, O
has O
, O
icc O
. O
Obeso O
, O
has O
, O
icc O
. O
Obeso O
, O
has O
, O
icc O
. O
c O
# O
cintilografia O
miocardica O
para O
avaliar O
angina B-ENT
. O
c O
# O
cintilografia B-ENT
miocardica I-ENT
para O
avaliar B-ENT
angina O
. O
c O
# O
cintilografia O
miocardica O
para O
avaliar O
angina O
. O
c O
# O
cintilografia O
miocardica B-ENT
para O
avaliar O
angina O
. O
Plastia O
Mitral O
( O
Insuficiencia B-ENT
) O
, O
CRM O
Saf O
- O
2Mg O
e O
e O
Saf O
- O
3MG O
) O
. O
Plastia O
Mitral O
( O
Insufici

DM B-ENT
, O
# O
HAS B-ENT
# O
DAC B-ENT
- O
iam B-ENT
em O
2009 O
- O
- O
> O
atc O
. O
LUCAS O
, O
57 O
ANOS O
# O
DM O
, O
# O
HAS O
# O
DAC O
- O
iam O
em O
2009 O
- O
- O
> O
atc O
. O
LUCAS O
, O
57 O
ANOS O
# O
DM O
, O
# O
HAS O
# O
DAC O
- O
iam O
em O
2009 O
- O
- O
> O
atc B-ENT
. O
LUCAS O
, O
57 O
ANOS O
# O
DM O
, O
# O
HAS O
# O
DAC O
- O
iam O
em O
2009 O
- O
- O
> O
atc O
. O
# O
RVM O
em O
2009 O
. O
# O
RVM O
em O
2009 O
. O
# O
RVM B-ENT
em O
2009 O
. O
# O
RVM O
em O
2009 O
. O
# O
ATC O
EM O
2013 O
DE O
CD O
+ O
IMPLANTE O
DE O
MP O
( O
bav B-ENT
de I-ENT
2 I-ENT
grau I-ENT
mobitz I-ENT
2 I-ENT
) O
# O
Dislipidemia B-ENT
. O
# O
ATC O
EM O
2013 O
DE O
CD O
+ O
IMPLANTE O
DE O
MP O
( O
bav O
de O
2 O
grau O
mobitz O
2 O
) O
# O
Dislipidemia O
. O
# O
ATC B-ENT
EM O
2013 O
DE O
CD O
+ O
IMPLANTE O
DE O
MP B-ENT
( O
bav O
de O
2 O
grau O
mobitz O
2 O
) O
# O
Dislipidemia O
. O
# O
ATC O
EM O
2013 O
DE O
CD B-ENT
+ O
IMPLANTE O
DE O
MP O
( O
bav O
de O
2 O
grau O
mobi

1 O
cp O
após O
almoço O
. O
Atualmente O
em O
uso O
furosemida O
40mg O
1 O
cp O
ao O
dia O
, O
digoxina O
0 O
, O
25mg O
1 O
cp O
/ O
dia O
, O
carvedilol O
25mg O
2 O
cp O
/ O
dia O
, O
monocordil O
20mg O
2 O
cp O
/ O
dia O
, O
hidralazina O
25mg O
2 O
cp O
/ O
dia O
, O
alopurinol O
300mg O
1 O
cp O
/ O
dia O
, O
colchicina O
0 O
, O
5mg O
se O
dor O
, O
marevan O
5mg O
2 O
cp O
seg O
, O
qua O
e O
sex O
, O
sab O
e O
1 O
cp O
nos O
demais O
, O
metformina O
850mg O
1 O
cp O
após O
almoço O
. O
HMP O
# O
Já O
esteve O
na O
fila O
para O
transplante O
- O
mretirado O
por O
melhora O
clinica O
. O
HMP O
# O
Já O
esteve O
na O
fila O
para O
transplante O
- O
mretirado O
por O
melhora O
clinica O
. O
HMP O
# O
Já O
esteve O
na O
fila O
para O
transplante B-ENT
- O
mretirado O
por O
melhora O
clinica O
. O
HMP O
# O
Já O
esteve O
na O
fila O
para O
transplante O
- O
mretirado O
por O
melhora O
clinica O
. O
CATE O
há O
aprox O
2 O
anos O
- O
sem O
evidência O
de O
doença B-ENT
coronari

ônibus O
aposentado O
, O
6 O
filhos O
. O
ID O
: O
Lucas O
, O
79 O
anos O
, O
viuvo O
, O
cobrador O
de O
ônibus O
aposentado O
, O
6 O
filhos O
. O
# O
Medicações O
em O
uso O
: O
- O
losartana O
50 O
mg O
12 O
/ O
12 O
h O
- O
AAS O
100 O
mg O
- O
captopril O
? O
1 O
x O
ao O
dia O
- O
hidroclorotiazida O
25 O
mg O
1 O
x O
ao O
dia O
- O
flutamida O
250 O
mg O
8 O
/ O
8 O
h O
# O
Comorbidades B-ENT
: O
HAS B-ENT
, O
Ca B-ENT
de I-ENT
Prostata I-ENT
em O
acompanhamento O
há O
2 O
anos O
. O
# O
Medicações O
em O
uso O
: O
- O
losartana O
50 O
mg O
12 O
/ O
12 O
h O
- O
AAS O
100 O
mg O
- O
captopril O
? O
1 O
x O
ao O
dia O
- O
hidroclorotiazida O
25 O
mg O
1 O
x O
ao O
dia O
- O
flutamida O
250 O
mg O
8 O
/ O
8 O
h O
# O
Comorbidades O
: O
HAS O
, O
Ca O
de O
Prostata O
em O
acompanhamento O
há O
2 O
anos O
. O
# O
Medicações B-ENT
em O
uso O
: O
- O
losartana B-ENT
50 I-ENT
mg I-ENT
12 O
/ O
12 O
h O
- O
AAS B-ENT
100 I-ENT
mg I-ENT
- O
captopril B-ENT
? O
1 O
x O
ao O
dia O
- O
h

Paciente O
refere O
que O
não O
houve O
mudança O
nos O
sintomas O
. O
# O
Relata O
que O
tem O
dispneia B-ENT
aos I-ENT
mínimos I-ENT
esforços I-ENT
. O
# O
Relata O
que O
tem O
dispneia O
aos O
mínimos O
esforços O
. O
# O
Relata O
que O
tem O
dispneia O
aos O
mínimos O
esforços O
. O
# O
Relata O
que O
tem O
dispneia O
aos O
mínimos O
esforços O
. O
ICC B-ENT
há O
6 O
- O
7 O
anos O
. O
ICC O
há O
6 O
- O
7 O
anos O
. O
ICC O
há O
6 O
- O
7 O
anos O
. O
ICC O
há O
6 O
- O
7 O
anos O
. O
Historia O
de O
TB B-ENT
há O
9 O
anos O
fez O
tratamento O
tendo O
sido O
curado O
. O
Historia O
de O
TB O
há O
9 O
anos O
fez O
tratamento O
tendo O
sido O
curado O
. O
Historia O
de O
TB O
há O
9 O
anos O
fez O
tratamento B-ENT
tendo O
sido O
curado O
. O
Historia O
de O
TB O
há O
9 O
anos O
fez O
tratamento O
tendo O
sido O
curado O
. O
Fez O
cirurgia O
de O
ponte O
de O
safena O
em O
2013 O
ficando O
internado O
68 O
dias O
( O
infecção B-ENT
hospitalar I-ENT
) O
necessitando O
de O
transfusão 

...

; O
mobioidade O
adequada O
e O
rgurgitação O
moderada O
a O
importante O
. O
Gra O
max O
de O
25 O
e O
medio O
de O
13 O
Area O
de O
2 O
. O
Gra O
max O
de O
25 O
e O
medio O
de O
13 O
Area O
de O
2 O
. O
Gra O
max O
de O
25 O
e O
medio O
de O
13 O
Area O
de O
2 O
. O
Gra O
max O
de O
25 O
e O
medio O
de O
13 O
Area O
de O
2 O
. O
VAo O
= O
cuspides B-ENT
calcificadas I-ENT
com O
mobilidade B-ENT
comprometida I-ENT
. O
VAo O
= O
cuspides O
calcificadas O
com O
mobilidade O
comprometida O
. O
VAo O
= O
cuspides O
calcificadas O
com O
mobilidade O
comprometida O
. O
VAo B-ENT
= O
cuspides O
calcificadas O
com O
mobilidade O
comprometida O
. O
Grad O
max O
de O
55 O
e O
medio O
de O
33 O
. O
Grad O
max O
de O
55 O
e O
medio O
de O
33 O
. O
Grad O
max O
de O
55 O
e O
medio O
de O
33 O
. O
Grad O
max O
de O
55 O
e O
medio O
de O
33 O
. O
Regurgitação B-ENT
moderada I-ENT
. O
Regurgitação O
moderada O
. O
Regurgitação O
moderada O
. O
Regurgitação O
moderada O
. O
VTri O
= O
abertura O
adeq

DA O
P O
ANTERIOR O
E O
ANTERO O
SEPTAL O
DISCRETA O
ISQUEMIA O
ASSOCIADA O
, O
. O
# O
CINTILO O
: O
FIBROSE O
PERSISTENTE O
APICE O
E O
MEDIO O
APICAL O
DA O
P O
ANTERIOR O
E O
ANTERO O
SEPTAL O
DISCRETA O
ISQUEMIA O
ASSOCIADA O
, O
. O
# O
CINTILO O
: O
FIBROSE O
PERSISTENTE O
APICE B-ENT
E I-ENT
MEDIO I-ENT
APICAL I-ENT
DA O
P B-ENT
ANTERIOR I-ENT
E I-ENT
ANTERO I-ENT
SEPTAL I-ENT
DISCRETA O
ISQUEMIA O
ASSOCIADA O
, O
. O
FE O
23 O
- O
28 O
% O
. O
FE O
23 O
- O
28 O
% O
. O
FE O
23 O
- O
28 O
% O
. O
FE O
23 O
- O
28 O
% O
. O
# O
CINTILOGRAFIA O
16 O
/ O
01 O
/ O
15 O
: O
HIPOPERFUSÃO B-ENT
ACENTUADA I-ENT
E I-ENT
PERSISTENTE I-ENT
NO I-ENT
APICE I-ENT
E I-ENT
MEDIO I-ENT
APICAL I-ENT
PAREDE I-ENT
ANTERIOR I-ENT
E I-ENT
ANTERO I-ENT
SEPTAL I-ENT
VE I-ENT
. O
# O
CINTILOGRAFIA B-ENT
16 O
/ O
01 O
/ O
15 O
: O
HIPOPERFUSÃO O
ACENTUADA O
E O
PERSISTENTE O
NO O
APICE O
E O
MEDIO O
APICAL O
PAREDE O
ANTERIOR O
E O
ANTERO O
SEPTAL O
VE O
. O
# O
CINTILOGRAFIA O
16 O
/ O
01 O
/ O
15 O
:

esquerdo I-ENT
e O
Câmaras B-ENT
direitas I-ENT
normais O
. O
Refluxo B-ENT
mitral I-ENT
discreto I-ENT
. O
Refluxo O
mitral O
discreto O
. O
Refluxo O
mitral O
discreto O
. O
Refluxo O
mitral B-ENT
discreto O
. O
# O
CINTILO O
FIBROSE B-ENT
PERSISTENTE I-ENT
APICE I-ENT
E I-ENT
MEDIO I-ENT
APICAL I-ENT
DA I-ENT
P I-ENT
ANTERIOR I-ENT
E I-ENT
ANTERO I-ENT
SEPTAL I-ENT
DISCRETA I-ENT
ISQUEMIA I-ENT
ASSOCIADA O
. O
# O
CINTILO B-ENT
FIBROSE O
PERSISTENTE O
APICE O
E O
MEDIO O
APICAL O
DA O
P O
ANTERIOR O
E O
ANTERO O
SEPTAL O
DISCRETA O
ISQUEMIA O
ASSOCIADA O
. O
# O
CINTILO O
FIBROSE O
PERSISTENTE O
APICE O
E O
MEDIO O
APICAL O
DA O
P O
ANTERIOR O
E O
ANTERO O
SEPTAL O
DISCRETA O
ISQUEMIA O
ASSOCIADA O
. O
# O
CINTILO O
FIBROSE O
PERSISTENTE O
APICE B-ENT
E I-ENT
MEDIO I-ENT
APICAL I-ENT
DA I-ENT
P I-ENT
ANTERIOR I-ENT
E I-ENT
ANTERO I-ENT
SEPTAL I-ENT
DISCRETA O
ISQUEMIA O
ASSOCIADA O
. O
FE O
23 O
- O
28 O
% O
. O
FE O
23 O
- O
28 O
% O
. O
FE O
23 O
- O
28 O
% O
. O
FE O
23 O
- O
28 

CARDÍACA I-ENT
REALIZADA O
EM O
2005 O
: O
3 B-ENT
PONTES I-ENT
DE I-ENT
SAFENA I-ENT
, O
COM O
HISTÓRIA O
DE O
INFARTO O
EM O
2005 O
. O
CX O
CARDÍACA B-ENT
REALIZADA O
EM O
2005 O
: O
3 O
PONTES O
DE O
SAFENA O
, O
COM O
HISTÓRIA O
DE O
INFARTO O
EM O
2005 O
. O
PACIENTE O
RELATA O
VERTIGEM B-ENT
MESMO I-ENT
SENTADA I-ENT
, O
NAO O
RELATA O
ANGINA B-ENT
, O
NEGA O
DISPNÉIA B-ENT
, O
RELATA O
EDEMA B-ENT
DE I-ENT
MMII I-ENT
. O
PACIENTE O
RELATA O
VERTIGEM O
MESMO O
SENTADA O
, O
NAO O
RELATA O
ANGINA O
, O
NEGA O
DISPNÉIA O
, O
RELATA O
EDEMA O
DE O
MMII O
. O
PACIENTE O
RELATA O
VERTIGEM O
MESMO O
SENTADA O
, O
NAO O
RELATA O
ANGINA O
, O
NEGA O
DISPNÉIA O
, O
RELATA O
EDEMA O
DE O
MMII O
. O
PACIENTE O
RELATA O
VERTIGEM O
MESMO O
SENTADA O
, O
NAO O
RELATA O
ANGINA O
, O
NEGA O
DISPNÉIA O
, O
RELATA O
EDEMA O
DE O
MMII B-ENT
. O
EM O
USO O
DE O
: O
ATENOLOL O
( O
1 O
CP O
) O
, O
CAPTOPRIL O
( O
2 O
CP O
) O
, O
FUROSEMIDA O
( O
1 O
CP O
) O
, O
SINVASTATINA O
( O
1 O
CP O
A O
NOIT

sudorese B-ENT
e O
escurecimento B-ENT
visual I-ENT
. O
pródromos O
: O
tontura O
e O
sudorese O
e O
escurecimento O
visual O
. O
pródromos O
: O
tontura O
e O
sudorese O
e O
escurecimento O
visual O
. O
pródromos O
: O
tontura O
e O
sudorese O
e O
escurecimento O
visual O
. O
Duração O
de O
segundos O
há O
3 O
minutos O
. O
Duração O
de O
segundos O
há O
3 O
minutos O
. O
Duração O
de O
segundos O
há O
3 O
minutos O
. O
Duração O
de O
segundos O
há O
3 O
minutos O
. O
Nega O
confusão B-ENT
mental I-ENT
ou O
salivação B-ENT
após O
. O
Nega O
confusão O
mental O
ou O
salivação O
após O
. O
Nega O
confusão O
mental O
ou O
salivação O
após O
. O
Nega O
confusão O
mental O
ou O
salivação O
após O
. O
Nega O
uso O
de O
medicação O
. O
Nega O
uso O
de O
medicação O
. O
Nega O
uso O
de O
medicação B-ENT
. O
Nega O
uso O
de O
medicação O
. O
Nega O
outros B-ENT
sintomas I-ENT
. O
Nega O
outros O
sintomas O
. O
Nega O
outros O
sintomas O
. O
Nega O
outros O
sintomas O
. O
HF O
- O
p O
cardiopat

In [31]:
# gerar arquivo treinamento
path='data_ner_qa_iobes'
f_train = open(path+r'\nested_ner_qa_train.conll', 'w', encoding='utf-8')
listaEntidades=getTiposEntidade()

print('Gravando em ', path)
num_entidade_total=0
num_entidade_train=0
num_entidade_dev=0
num_entidade_test=0


for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    for tipoEntidade in listaEntidades:
        indiceEnts=[]
        isEntidade=0
        #f_train.write(tipoEntidade+' O\n[SEP]\n')
        f_train.write(tipoEntidade+' <sep>\n')
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            if tag!=tipoEntidade:
                tag='O'
                isEntidade=0
            else:
                if (isEntidade==1):
                    tag='I-ENT'
                else:
                    tag='B-ENT'
                isEntidade=1
            f_train.write(tokenGravar+' '+tag+'\n')
            num_entidade_train=num_entidade_train+1
        f_train.write('\n')        
f_train.close()

f_dev = open(path+r'\nested_ner_qa_dev.conll', 'w', encoding='utf-8')
for i in range(len(dic_sentencesDev)):
    tokens = dic_sentencesDev[i][0]
    ents = dic_sentencesDev[i][1]
    for tipoEntidade in listaEntidades:
        indiceEnts=[]
        isEntidade=0
        f_dev.write(tipoEntidade+' <sep>\n')
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            if tag!=tipoEntidade:
                tag='O'
                isEntidade=0
            else:
                if (isEntidade==1):
                    tag='I-ENT'
                else:
                    tag='B-ENT'
                isEntidade=1

            f_dev.write(tokenGravar+' '+tag+'\n')
            num_entidade_dev=num_entidade_dev+1
        f_dev.write('\n')
f_dev.close()


f_test = open(path+r'\nested_ner_qa_test.conll', 'w', encoding='utf-8')
for i in range(len(dic_sentencesTest)):
    tokens = dic_sentencesTest[i][0]
    ents = dic_sentencesTest[i][1]
    indiceEnts=[]
    isEntidade=0
    for tipoEntidade in listaEntidades:
        f_test.write(tipoEntidade+' <sep>\n')
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==tipoEntidade:
                    tag = ent[2]
                    break
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            if tag!=tipoEntidade:
                tag='O'
                isEntidade=0
            else:
                if (isEntidade==1):
                    tag='I-ENT'
                else:
                    tag='B-ENT'
                isEntidade=1

            f_test.write(tokenGravar+' '+tag+'\n')
            num_entidade_test=num_entidade_test+1
        f_test.write('\n')
f_test.close()

print('num_entidade_train', num_entidade_train)
print('num_entidade_dev', num_entidade_dev)
print('num_entidade_test:', num_entidade_test)
num_entidade_total=num_entidade_train+num_entidade_dev+num_entidade_test
print('num_entidade_total:', num_entidade_total)

#save_obj('dic_sentencesTrain',dic_sentencesTrain)
#save_obj('dic_sentencesDev',dic_sentencesDev)
#save_obj('dic_sentencesTest',dic_sentencesTest)


Gravando em  data_ner_qa_iobes
num_entidade_train 55296
num_entidade_dev 16040
num_entidade_test: 21852
num_entidade_total: 93188
